In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [ ]:
from google.colab import files
files.upload()

Saving makeHistoricalData.py to makeHistoricalData.py


{'makeHistoricalData.py': b'import pandas as pd\nimport numpy as np\n\n# h is the number of days before day (t)\n# r indicates how many days after day (t) --> target-day = day(t+r)\n# target could be number of deaths or number of confirmed \ndef makeHistoricalData(h, r, target, feature_selection):\n        \'\'\' in this code when h is 1, it means there is no history and we have just one column for each covariate\n        so when h is 0, we put h equal to 1, because when h is 0 that means there no history (as when h is 1) \'\'\'\n        if h == 0:\n                h = 1\n        \n        independantOfTimeData = pd.read_csv(\'/content/drive/My Drive/full-fixed-data.csv\')\n        timeDeapandantData = pd.read_csv(\'/content/drive/My Drive/full-temporal-data.csv\')\n\n        allData = pd.merge(independantOfTimeData, timeDeapandantData, on=\'county_fips\')\n        allData = allData.sort_values(by=[\'date\', \'county_fips\'])\n        allData = allData.reset_index(drop=True)\n        #

In [ ]:
!rm -rf Results/
!mkdir Results

!rm -rf Plots/
!mkdir Plots

In [ ]:
import makeHistoricalData

r = 14
target = 'confirmed'

for h in range(1, 8):
    data = makeHistoricalData.makeHistoricalData(h, r, target, 'mrmr')
    mean_error = 0
    errors = pd.DataFrame()

    states = ['Colorado', 'Minnesota', 'New York', 'Utah']
    # states = ['New York']
    for i in range(len(states)):
        temporal = data.loc[data['state_name'] == states[i]]    # we construct the (train, val, test) dataframes from temporal dataframe
        temporal.reset_index(drop=True, inplace=True)

        totalNumberOfDays = len(temporal['date of day t'].unique())
        totalNumberOfCounties = len(temporal.county_fips.unique())

        last14days = temporal.tail(14*totalNumberOfCounties)
        nameOfCountiesInLast14Days = last14days['county_fips']

        del temporal['date of day t']
        del temporal['county_name']
        del temporal['state_name']
        del temporal['state_fips']
        del temporal['county_fips']

        # last 14 days are for test dataset and the rest are for train dataset
        train = temporal.head((totalNumberOfDays-28)*totalNumberOfCounties)
        validation = temporal.loc[(totalNumberOfDays-28)*totalNumberOfCounties:(totalNumberOfDays-15)*totalNumberOfCounties, :]
        test = temporal.tail(14*totalNumberOfCounties)

        print(len(validation), len(train), len(test))

        X_train = train.drop('Target', axis=1)
        y_train = train['Target']

        X_val = validation.drop('Target', axis=1)
        y_val = validation['Target']

        X_test = test.drop('Target', axis=1)
        y_test = test['Target']

        ####################################### deleting some first days because of many zeros (here first 28 days)
        # X_train = X_train.iloc[28*totalNumberOfCounties:]
        # y_train = y_train.iloc[28*totalNumberOfCounties:]
        #######################################

        ################################################### Noramlizing the data
        scalar = preprocessing.MinMaxScaler()

        X_train = X_train.values
        X_train = scalar.fit_transform(X_train)

        X_val = X_val.values
        X_val = scalar.fit_transform(X_val)

        X_test = X_test.values
        X_test = scalar.fit_transform(X_test)

        y_train = y_train.values
        y_train = scalar.fit_transform(y_train.reshape(-1, 1))

        y_val = y_val.values
        y_val = scalar.fit_transform(y_val.reshape(-1, 1))

        # y_test = y_test.values
        # min_max_scaler = preprocessing.MinMaxScaler()
        # y_test = min_max_scaler.fit_transform(y_test.reshape(-1, 1))
        ################################################### end of normalazing

        X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
        X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

        y_train = y_train.reshape((y_train.shape[0]), )
        y_val = y_val.reshape((y_val.shape[0]), )
        # y_test = y_test.reshape((y_test.shape[0]), )

        print('Working on state = ', states[i], ' and h = ', h)

        print(X_train.shape, X_val.shape, X_test.shape)
        print(y_train.shape, y_val.shape, y_test.shape)
        
        ############################################# defining the model
        model = Sequential()
        model.add(LSTM(8, return_sequences=True, input_shape=(1, X_train.shape[2])))
        model.add(LSTM(256, dropout=0.2, return_sequences=True))
        model.add(LSTM(256, dropout=0.2, return_sequences=True))
        model.add(LSTM(128, dropout=0.2, return_sequences=True))
        model.add(LSTM(128))
        model.add(Dense(1, activation='selu'))

        # opt = tf.keras.optimizers.RMSprop(learning_rate=0.1)
        model.compile (
            loss='mean_squared_error',
            optimizer=keras.optimizers.Adam(0.01)
        )
        ############################################# end of defining the model

        history = model.fit (
            X_train, y_train,
            epochs=50,
            batch_size=128,
            validation_data=(X_val, y_val),
            verbose=1,
            shuffle=False
        )

        # getting the result on test dataset
        pred = model.predict(X_test)
        pred = scalar.inverse_transform(pred.reshape(-1, 1))
        # y_test = scalar.inverse_transform(y_test.reshape(-1, 1))

        y_test = y_test.to_numpy()
        y_test = y_test.reshape(-1, 1)
        pred = pred.reshape(-1, 1)
        pred = np.round(pred)

        ##################################### storing predictinos in a file
        result = pd.DataFrame()
        result['county_fips'] = nameOfCountiesInLast14Days.tolist()
        result['Prediction'] = pred.tolist()
        result['Actual'] = y_test.tolist()
        file_name = '/content/Results/predictions_'+str(states[i])+'_'+str(h)+'.csv'
        result.to_csv(file_name, mode='w', index=False)
        #####################################

        ##################################### ploting y_test and pred in a same graph
        days = [x for x in range(1,14*(totalNumberOfCounties)+1)]
        fig = plt.figure(figsize=(35, 10), dpi=600)
        plt.plot(days, y_test.tolist(), label='Actual')
        plt.plot(days, pred.tolist(), label='Prediction')
        plt.xlabel("Day")
        plt.ylabel("Target")
        plt.legend()
        fig_name = '/content/Plots/'+str(states[i])+'_'+str(h)+'.png'
        fig.savefig(fig_name, dpi=600)
        #####################################

        pred[pred < 0] = 0
        pred[pred == 0] += 1
        y_test = np.round(y_test)
        y_test[y_test == 0] = 1

        MSE = (np.square(y_test - pred)).mean(axis=None)
        RMSE = np.sqrt(MSE)        
        MAPE = np.mean(np.abs((y_test - pred) / y_test)) * 100

        r2 = r2_score(y_test, pred)

        key = str(states[i])+'_'+str(h)
        temp_list = [[key, MSE, RMSE, MAPE, r2]]
        errors = errors.append(pd.DataFrame(temp_list, columns=['county and h', 'MSE','RMSE', 'MAPE', 'adj_r_squared']), ignore_index=True)

    num += 1
    file_name = '/content/Results/errors_per_state_h='+str(num)+'.csv'
    errors.to_csv(file_name, mode='w', index=False)

742 6213 798
Working on state =  Colorado  and h =  1
(6213, 1, 39) (742, 1, 39) (798, 1, 39)
(6213,) (742,) (798,)
Train on 6213 samples, validate on 742 samples
Epoch 1/50
6213/6213 [==============================] - 4s 660us/step - loss: 7.3485e-04 - val_loss: 0.0126
Epoch 2/50
6213/6213 [==============================] - 2s 338us/step - loss: 6.2234e-04 - val_loss: 0.0102
Epoch 3/50
6213/6213 [==============================] - 2s 325us/step - loss: 6.6918e-04 - val_loss: 0.0099
Epoch 4/50
6213/6213 [==============================] - 2s 346us/step - loss: 6.3744e-04 - val_loss: 0.0130
Epoch 5/50
6213/6213 [==============================] - 2s 343us/step - loss: 6.0926e-04 - val_loss: 0.0072
Epoch 6/50
6213/6213 [==============================] - 2s 350us/step - loss: 6.3224e-04 - val_loss: 0.0107
Epoch 7/50
6213/6213 [==============================] - 2s 340us/step - loss: 5.0520e-04 - val_loss: 0.0087
Epoch 8/50
6213/6213 [==============================] - 2s 344us/step - loss: 5.8

In [ ]:
# run this cell to make zip file from results and then download them :)

!zip -r /content/Results.zip /content/Results/
!zip -r /content/Plots.zip /content/Plots/

updating: content/Results/ (stored 0%)
updating: content/Results/errors_per_state_h=7.csv (deflated 9%)
updating: content/Results/predictions_New York_6.csv (deflated 87%)
updating: content/Results/errors_per_state_h=6.csv (deflated 9%)
updating: content/Results/errors_per_state_h=4.csv (deflated 9%)
updating: content/Results/errors_per_state_h=5.csv (deflated 10%)
updating: content/Results/predictions_New York_1.csv (deflated 87%)
updating: content/Results/predictions_New York_7.csv (deflated 87%)
updating: content/Results/errors_per_state_h=1.csv (deflated 9%)
updating: content/Results/predictions_New York_5.csv (deflated 84%)
updating: content/Results/errors_per_state_h=2.csv (deflated 9%)
updating: content/Results/predictions_New York_3.csv (deflated 86%)
updating: content/Results/predictions_New York_4.csv (deflated 87%)
updating: content/Results/errors_per_state_h=3.csv (deflated 9%)
updating: content/Results/predictions_New York_2.csv (deflated 87%)
updating: content/Plots/ (sto